In [22]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo


import os

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.docstore.document import Document
from langchain.vectorstores import Qdrant
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

from langchain.chat_models import ChatOpenAI

os.environ["OPENAI_API_KEY"] = "random-string"


# Using sentence transformers all-MiniLM-L6-v2
embeddings = OpenAIEmbeddings(openai_api_base="http://localhost:8444/v1")

# Qdrant client
client = QdrantClient("localhost", port=6333)
qdrant_docsearch = Qdrant(client=client, collection_name="recipe_title_collection", embeddings=embeddings, content_payload_key="title")

# chat completion llm
llm = ChatOpenAI(
    openai_api_base="http://localhost:8111/v1", max_tokens=128
)


# Using OpenAI directly
# qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=qdrant_docsearch.as_retriever())

# Defining metadata fields
metadata_field_info = [
    AttributeInfo(
        name="recipe",
        description="The recipe of the dish",
        type="string",
    ),
    AttributeInfo(
        name="image",
        description="The file name of the image",
        type="string",
    ),
]
document_content_description = "Title of the recipe"

# Using SelfQueryRetriever
retriever = SelfQueryRetriever.from_llm(
    llm, qdrant_docsearch, document_content_description, metadata_field_info, verbose=True, enable_limit=True, 
)

# print(retriever.get_relevant_documents("any recipes with chicken and leek"))

# Using Vicuna via Premai app 
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True)

query = "what recipes do you have with chicken and leek?"
result = qa({"query": query})

print(result)

/Users/aditya/anaconda3/envs/aditya/lib/python3.11/site-packages/langchain/chains/llm.py:275: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


query='chicken leek' filter=None limit=None
{'query': 'what recipes do you have with chicken and leek?', 'result': 'I have two recipes that include chicken and leeks: Grilled Lemon-Garlic Chicken with Leeks and Potatoes and Lemon-Rhubarb Chicken.', 'source_documents': [Document(page_content='Beef Stew with Leeks', metadata={}), Document(page_content='Creamy Leek Soup', metadata={}), Document(page_content='Grilled Lemon-Garlic Chicken with Leeks and Potatoes', metadata={}), Document(page_content='Lemon-Rhubarb Chicken', metadata={})]}
